In [1]:
# univariate multi-step lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM

In [2]:
# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
# 	train, test = data[1:-328], data[-328:-6]
    train = data[data['NewDateTime']>= '2021-01-01'].copy()
    train = data_training[data_training['NewDateTime']< '2021-10-08'].copy()
    test = data[data['NewDateTime']>= '2021-10-08'].copy()
	# restructure into windows of weekly data
    print(len(train))
    print(len(test))
	train = array(split(train, len(train)/7))
	test = array(split(test, len(test)/7))
	return train, test

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-2-ebee422ed796>, line 11)

In [3]:
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores


In [5]:
#  summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end < len(data):
			x_input = data[in_start:in_end, 0]
			x_input = x_input.reshape((len(x_input), 1))
			X.append(x_input)
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

# train the model
def build_model(train, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 0, 70, 16
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# define model
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mse', optimizer='adam')
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, 0]
	# reshape into [1, n_input, 1]
	input_x = input_x.reshape((1, len(input_x), 1))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

# evaluate a single model
def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores


In [29]:
# univariate multi-step lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM, Dropout

In [30]:
# load the new file
dataset = read_csv('./data/bitcoin_data.csv', date_parser = True)
dataset.drop(dataset.columns[0], axis=1,inplace = True)
dataset.tail()


,close,open,high,low,Volume BTC,NewDateTime,returns,log_returns,SOPR,fundingRate,googleTrend
19428,50884.48,50921.61,51204.77,50666.00,1312.93360,2021-12-07 19:00:00,0.999271,-0.000729,1.004981,0.0039,23
19429,50441.59,50884.49,51006.20,50150.00,2194.52091,2021-12-07 20:00:00,0.991296,-0.008742,1.004981,0.0039,23
19430,50489.92,50441.59,50856.46,50358.99,1468.15088,2021-12-07 21:00:00,1.000958,0.000958,1.004981,0.0039,23
19431,50245.25,50489.93,50626.43,50039.74,1362.13177,2021-12-07 22:00:00,0.995154,-0.004858,1.004981,0.0039,23
19432,50588.95,50245.25,50710.99,50141.60,828.39608,2021-12-07 23:00:00,1.006840,0.006817,1.004981,0.0039,22


In [31]:
# split into train and test
PDate = '2021-10-08'
train = dataset[dataset['NewDateTime']>= '2021-01-01'].copy()
train = train[train['NewDateTime']< PDate].copy()
test = dataset[dataset['NewDateTime']>= PDate].copy()
# restructure into windows of weekly data

train.drop(train.head(len(train)%168).index,inplace=True)
train = train.drop(['NewDateTime'], axis=1)
test.drop(test.tail(len(test)%168).index,inplace=True)
test = test.drop(['NewDateTime'], axis=1)
train = array(split(train, len(train)/24))
# print(len(test))

test = array(split(test, len(test)/24))


In [32]:
# evaluate model and get scores
n_input = 168
import numpy as np
train = np.asarray(train).astype(np.float32)
test = np.asarray(test).astype(np.float32)

In [33]:

# history is a list of weekly data
# prepare data
# train_x, train_y = to_supervised(train, n_input)
n_out=24
# flatten data
data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
X, y = list(), list()
in_start = 0
# step over the entire history one time step at a time
for _ in range(len(data)):
	# define the end of the input sequence
	in_end = in_start + n_input
	out_end = in_end + n_out
	# ensure we have enough data for this instance
	if out_end < len(data):
		X.append(data[in_start:in_end, :])
		y.append(data[in_end:out_end, 0])
	# move along one time step
	in_start += 1
train_x ,train_y =array(X), array(y)




In [1]:
train_x[0,0,0]

NameError: name 'train_x' is not defined

In [35]:
train_y.shape

(6360, 24)

In [42]:
# define parameters
verbose, epochs, batch_size = 0, 70, 32
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(Dropout(0.1))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(n_outputs))

model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 200)               168800    
_________________________________________________________________
dropout_8 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 24)                2424      
Total params: 191,324
Trainable params: 191,324
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.compile(loss='mse', optimizer='adam')
# fit network
model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size)

Epoch 1/70
 19/199 [=>............................] - ETA: 3:00 - loss: nan

KeyboardInterrupt: 

In [39]:
history = [x for x in train]
# walk-forward validation over each week
predictions = list()
for i in range(len(test)):
    # predict the week
    # flatten data
    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
    # retrieve last observations for input data
    input_x = data[-n_input:, 0]
    # reshape into [1, n_input, 1]
    input_x = input_x.reshape((1, len(input_x), 1))
    # forecast the next week
    yhat = model.predict(input_x, verbose=0)
    # we only want the vector forecast
    yhat = yhat[0]
    # store the predictions
    predictions.append(yhat)
    # get real observation and add to history for predicting the next week
    history.append(test[i, :])
# evaluate predictions days for each week
predictions = array(predictions)



ValueError: in user code:

    /opt/conda/lib/python3.6/site-packages/keras/engine/training.py:1586 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.6/site-packages/keras/engine/training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.6/site-packages/keras/engine/training.py:1569 run_step  **
        outputs = model.predict_step(data)
    /opt/conda/lib/python3.6/si

In [ ]:
actual = test[:, :, 0]
predicted = predictions
scores = list()
# calculate an RMSE score for each day
for i in range(actual.shape[1]):
	# calculate mse
	mse = mean_squared_error(actual[:, i], predicted[:, i])
	# calculate rmse
	rmse = sqrt(mse)
	# store
	scores.append(rmse)
# calculate overall RMSE
s = 0
for row in range(actual.shape[0]):
	for col in range(actual.shape[1]):
		s += (actual[row, col] - predicted[row, col])**2
score = sqrt(s / (actual.shape[0] * actual.shape[1]))

In [ ]:
# summarize scores
summarize_scores('lstm', score, scores)

In [ ]:
# plot scores
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
pyplot.plot(days, scores, marker='o', label='lstm')
pyplot.show()